In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./eeg-motor-imagery-bciciv-2a/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./eeg-motor-imagery-bciciv-2a/.DS_Store
./eeg-motor-imagery-bciciv-2a/BCICIV_2a_all_patients.csv
./eeg-motor-imagery-bciciv-2a/patients/.DS_Store
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_1.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_3.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_2.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_6.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_7.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_5.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_4.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_9.csv
./eeg-motor-imagery-bciciv-2a/patients/patients/BCICIV_2a_8.csv


In [25]:
df=pd.read_csv("./eeg-motor-imagery-bciciv-2a/BCICIV_2a_all_patients.csv")

In [26]:
df

,patient,time,label,epoch,EEG-Fz,EEG-0,EEG-1,EEG-2,EEG-3,EEG-4,...,EEG-8,EEG-9,EEG-10,EEG-11,EEG-12,EEG-13,EEG-14,EEG-Pz,EEG-15,EEG-16
0,1,-0.100,tongue,8,-1.681412,2.245496,-0.158350,1.163765,-1.523659,-0.575267,...,0.758116,3.441785,0.305517,1.137473,-1.275763,-2.898359,0.656704,-2.010063,-1.613804,-1.942455
1,1,-0.096,tongue,8,0.420417,0.587559,1.650510,0.970672,1.505904,0.891796,...,1.541586,-0.071620,0.258909,-1.448198,0.142472,-1.968405,-1.733655,-2.935578,-3.125256,-4.674610
2,1,-0.092,tongue,8,0.551365,1.499758,0.121302,2.859433,2.613414,4.636026,...,2.649097,-2.137938,-1.612096,-1.610218,-0.410173,-0.274957,-4.776535,-5.099551,-2.798995,-5.862021
3,1,-0.088,tongue,8,3.054916,-1.807238,1.843603,2.286812,5.995872,6.651295,...,6.031554,-5.249621,-2.672998,-3.452370,0.189081,1.593829,-6.081577,-5.476860,-2.932163,-6.874095
4,1,-0.084,tongue,8,2.506710,-2.453101,0.221178,0.127278,4.519931,6.249573,...,7.827097,-5.309546,-2.488783,-3.707608,1.447515,4.268278,-4.383690,-4.218426,-1.331932,-5.322692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492043,9,0.684,foot,638,7.375522,4.646781,3.071135,4.665561,1.087962,0.932088,...,-2.314982,2.922773,-3.176987,-4.865313,-1.569414,3.600733,-1.113059,-4.249327,-1.402272,-6.307621
492044,9,0.688,foot,638,7.268988,1.952357,5.503664,3.484809,3.227522,-2.738899,...,-2.665657,4.085770,-0.500318,-5.655441,-1.871261,2.517636,-1.805531,-4.453518,-1.801775,-5.974702
492045,9,0.692,foot,638,4.581222,3.463809,4.427226,4.752120,1.906943,-2.496978,...,-5.206939,6.915582,-1.088475,-4.534613,-3.094184,1.148229,-2.247203,-4.895190,-1.803994,-5.879265
492046,9,0.696,foot,638,3.504784,4.096355,2.960162,3.187401,2.148864,-3.182791,...,-5.306815,6.620394,-1.286006,-5.806364,-2.705778,2.269057,-0.540439,-3.530222,-0.194886,-1.486953


In [27]:
df.shape

(492048, 26)

In [28]:
df.describe()

,patient,time,epoch,EEG-Fz,EEG-0,EEG-1,EEG-2,EEG-3,EEG-4,EEG-5,...,EEG-8,EEG-9,EEG-10,EEG-11,EEG-12,EEG-13,EEG-14,EEG-Pz,EEG-15,EEG-16
count,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,...,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000,492048.000000
mean,5.058824,0.300000,312.133578,-0.811991,-0.399268,-0.669511,-0.908202,-0.607526,-0.325437,0.161790,...,0.204116,0.503087,0.305026,0.198725,0.255002,0.403190,0.741585,0.656315,0.618452,0.770249
std,2.645100,0.232092,176.757760,7.345682,5.758001,5.182636,5.864248,5.159808,5.475433,6.364779,...,6.205865,5.213024,3.651272,3.858260,3.715157,5.382023,5.700428,5.596708,5.853360,8.009687
min,1.000000,-0.100000,4.000000,-83.147212,-41.675317,-33.485423,-47.021740,-38.127339,-46.884049,-48.302710,...,-59.941901,-44.244171,-26.766605,-34.876342,-34.383536,-80.331997,-44.953972,-45.883926,-50.665326,-63.052646
25%,3.000000,0.100000,159.750000,-4.968283,-3.791030,-3.677560,-4.254897,-3.664436,-3.591620,-3.696852,...,-3.477809,-2.334381,-1.782227,-2.022270,-1.876298,-2.423160,-2.612049,-2.568257,-2.790032,-3.918116
50%,5.000000,0.300000,312.000000,-0.657985,-0.315676,-0.554353,-0.732763,-0.513890,-0.295103,0.155490,...,0.172776,0.518842,0.288530,0.194630,0.223781,0.400869,0.622900,0.544408,0.516750,0.615729
75%,7.000000,0.500000,465.000000,3.458858,3.057477,2.433318,2.581403,2.498685,2.943858,4.040869,...,3.859897,3.433846,2.412297,2.474890,2.396761,3.280874,4.007513,3.831749,3.953051,5.326128
max,9.000000,0.700000,656.000000,56.453337,51.751069,39.955918,40.397420,32.779037,38.145863,68.064016,...,67.502066,45.002544,40.933676,24.452903,25.866699,40.688767,42.318192,41.931835,45.042494,67.761316


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492048 entries, 0 to 492047
Data columns (total 26 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   patient  492048 non-null  int64  
 1   time     492048 non-null  float64
 2   label    492048 non-null  object 
 3   epoch    492048 non-null  int64  
 4   EEG-Fz   492048 non-null  float64
 5   EEG-0    492048 non-null  float64
 6   EEG-1    492048 non-null  float64
 7   EEG-2    492048 non-null  float64
 8   EEG-3    492048 non-null  float64
 9   EEG-4    492048 non-null  float64
 10  EEG-5    492048 non-null  float64
 11  EEG-C3   492048 non-null  float64
 12  EEG-6    492048 non-null  float64
 13  EEG-Cz   492048 non-null  float64
 14  EEG-7    492048 non-null  float64
 15  EEG-C4   492048 non-null  float64
 16  EEG-8    492048 non-null  float64
 17  EEG-9    492048 non-null  float64
 18  EEG-10   492048 non-null  float64
 19  EEG-11   492048 non-null  float64
 20  EEG-12   492048 non-null  

In [30]:
df.columns

Index(['patient', 'time', 'label', 'epoch', 'EEG-Fz', 'EEG-0', 'EEG-1',
       'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-Cz',
       'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 'EEG-10', 'EEG-11', 'EEG-12',
       'EEG-13', 'EEG-14', 'EEG-Pz', 'EEG-15', 'EEG-16'],
      dtype='object')

In [31]:
df['label'].unique()

array(['tongue', 'foot', 'right', 'left'], dtype=object)

* 0: Hand Front
* 1: Hand Back
* 2: Bend
* 3: Grab

In [32]:
df['label'].replace(['tongue','foot','right','left'],[0,1,2,3],inplace=True)

/var/folders/wv/qhng4z0d725grffz0txv7yth0000gn/T/ipykernel_14046/2294284718.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].replace(['tongue','foot','right','left'],[0,1,2,3],inplace=True)
/var/folders/wv/qhng4z0d725grffz0txv7yth0000gn/T/ipykernel_14046/2294284718.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.

In [33]:
df

,patient,time,label,epoch,EEG-Fz,EEG-0,EEG-1,EEG-2,EEG-3,EEG-4,...,EEG-8,EEG-9,EEG-10,EEG-11,EEG-12,EEG-13,EEG-14,EEG-Pz,EEG-15,EEG-16
0,1,-0.100,0,8,-1.681412,2.245496,-0.158350,1.163765,-1.523659,-0.575267,...,0.758116,3.441785,0.305517,1.137473,-1.275763,-2.898359,0.656704,-2.010063,-1.613804,-1.942455
1,1,-0.096,0,8,0.420417,0.587559,1.650510,0.970672,1.505904,0.891796,...,1.541586,-0.071620,0.258909,-1.448198,0.142472,-1.968405,-1.733655,-2.935578,-3.125256,-4.674610
2,1,-0.092,0,8,0.551365,1.499758,0.121302,2.859433,2.613414,4.636026,...,2.649097,-2.137938,-1.612096,-1.610218,-0.410173,-0.274957,-4.776535,-5.099551,-2.798995,-5.862021
3,1,-0.088,0,8,3.054916,-1.807238,1.843603,2.286812,5.995872,6.651295,...,6.031554,-5.249621,-2.672998,-3.452370,0.189081,1.593829,-6.081577,-5.476860,-2.932163,-6.874095
4,1,-0.084,0,8,2.506710,-2.453101,0.221178,0.127278,4.519931,6.249573,...,7.827097,-5.309546,-2.488783,-3.707608,1.447515,4.268278,-4.383690,-4.218426,-1.331932,-5.322692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492043,9,0.684,1,638,7.375522,4.646781,3.071135,4.665561,1.087962,0.932088,...,-2.314982,2.922773,-3.176987,-4.865313,-1.569414,3.600733,-1.113059,-4.249327,-1.402272,-6.307621
492044,9,0.688,1,638,7.268988,1.952357,5.503664,3.484809,3.227522,-2.738899,...,-2.665657,4.085770,-0.500318,-5.655441,-1.871261,2.517636,-1.805531,-4.453518,-1.801775,-5.974702
492045,9,0.692,1,638,4.581222,3.463809,4.427226,4.752120,1.906943,-2.496978,...,-5.206939,6.915582,-1.088475,-4.534613,-3.094184,1.148229,-2.247203,-4.895190,-1.803994,-5.879265
492046,9,0.696,1,638,3.504784,4.096355,2.960162,3.187401,2.148864,-3.182791,...,-5.306815,6.620394,-1.286006,-5.806364,-2.705778,2.269057,-0.540439,-3.530222,-0.194886,-1.486953


In [34]:
df['patient'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [35]:
patient_dataframes = {}

for patient in df['patient'].unique():
    patient_df = df[df['patient'] == patient]
    patient_dataframes[patient] = patient_df

In [36]:
patient_dataframes[4]

,patient,time,label,epoch,EEG-Fz,EEG-0,EEG-1,EEG-2,EEG-3,EEG-4,...,EEG-8,EEG-9,EEG-10,EEG-11,EEG-12,EEG-13,EEG-14,EEG-Pz,EEG-15,EEG-16
173664,4,-0.100,2,4,-2.394456,3.232046,2.043269,1.387846,0.585938,-0.647912,...,0.599084,-1.031025,-0.281701,-0.268555,-0.794396,-1.329627,-3.756010,-2.743765,-2.244216,-6.244366
173665,4,-0.096,2,4,-4.265461,-0.347943,0.367577,0.688716,-0.796786,-0.712276,...,3.610891,-0.997733,-0.980831,1.962003,1.387333,1.438039,-2.892640,-1.733911,-0.843736,-3.330215
173666,4,-0.092,2,4,-2.403334,2.881372,2.473845,1.183655,0.674716,0.954539,...,-0.044560,-2.504746,-0.827688,0.503817,0.515086,0.565792,-2.202387,-0.653033,-1.227703,-4.934884
173667,4,-0.088,2,4,-1.662034,-1.943735,2.287410,-1.297701,1.904297,-1.868615,...,1.575646,-2.495868,0.890174,1.000977,1.988807,0.574669,-1.949369,0.332407,0.392503,-2.240460
173668,4,-0.084,2,4,-3.533039,2.142291,2.076561,-1.996831,-0.064364,-2.665401,...,-0.100046,-0.997733,1.607060,1.131925,0.996708,-0.515086,-1.525452,-0.317895,-0.501939,-2.304824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202603,4,0.684,3,603,-1.889614,1.582817,-1.776934,-1.273629,0.591230,2.933102,...,1.225996,1.406284,-2.458650,-2.276483,-0.988172,2.294580,0.625034,-1.701814,0.566816,0.465404
202604,4,0.688,3,603,-0.520207,1.389724,-1.921199,-1.613206,0.837590,3.032978,...,0.642278,-0.105168,-2.505258,-2.860201,-0.009390,3.126878,2.629207,-0.869516,1.838567,1.932467
202605,4,0.692,3,603,1.430698,0.801567,0.127363,-2.982613,1.225996,1.419430,...,2.300214,-1.035122,-1.091462,-2.618280,-1.427625,1.171534,1.845737,-1.067048,2.275800,2.467357
202606,4,0.696,3,603,3.334995,0.557426,1.494550,-1.322457,3.960371,1.321774,...,3.765058,-2.451138,-1.775056,-2.813593,-2.208875,0.194971,0.429722,-1.799470,1.738691,2.076732


In [37]:
df.isna().sum()

patient    0
time       0
label      0
epoch      0
EEG-Fz     0
EEG-0      0
EEG-1      0
EEG-2      0
EEG-3      0
EEG-4      0
EEG-5      0
EEG-C3     0
EEG-6      0
EEG-Cz     0
EEG-7      0
EEG-C4     0
EEG-8      0
EEG-9      0
EEG-10     0
EEG-11     0
EEG-12     0
EEG-13     0
EEG-14     0
EEG-Pz     0
EEG-15     0
EEG-16     0
dtype: int64

In [43]:
X = df.drop(columns=['label', 'patient'])
y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [61]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
batch_size = 32
num_microbatches = 32
learning_rate = 0.25

In [62]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # Assuming 4 classes (0, 1, 2, 3)

In [68]:
optimizer = DPGradientDescentGaussianOptimizer(
    l2_norm_clip=1.0,      # Gradient clipping norm
    noise_multiplier=0.5,  # Controls privacy strength (lower = more privacy)
    num_microbatches=None, # Use per-example gradients
    learning_rate=0.001
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction=tf.losses.Reduction.NONE)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [69]:
history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_split=0.2)

Train on 314910 samples, validate on 78728 samples
Epoch 1/50
  1824/314910 [..............................] - ETA: 24s - loss: 39.8598 - acc: 0.2423

2025-04-27 19:01:20.935707: W tensorflow/c/c_api.cc:305] Operation '{name:'count_8/Assign' id:3012 op device:{requested: '', assigned: ''} def:{{{node count_8/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](count_8, count_8/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


314368/314910 [============================>.] - ETA: 0s - loss: 32.5816 - acc: 0.2527

ValueError: operands could not be broadcast together with shapes (32,) (30,) (32,) 

In [40]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [20]:
# new_data = X_test[:20] 
# print(new_data)
predictions = model.predict(X_test)
# print(predictions)

predicted_classes = predictions.argmax(axis=1)
# print(predicted_classes)

# movement={0:"Hand Front",1:"Hand Back",2:"Bend",3:"Grab"}

# for i in predicted_classes:
#     print(movement[i])

/opt/anaconda3/envs/zazu2/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2025-04-27 18:22:31.314907: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_2/Softmax' id:99 op device:{requested: '', assigned: ''} def:{{{node dense_2/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_2/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [21]:

model.save("./models/hand_model.h5")

/opt/anaconda3/envs/zazu2/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loaded_model=tf.keras.models.load_model('./models/hand_model.h5')

In [23]:
print(classification_report(predicted_classes, y_test))

              precision    recall  f1-score   support

           0       0.17      0.41      0.24      9633
           1       0.15      0.43      0.23      8256
           2       0.73      0.31      0.44     60215
           3       0.29      0.38      0.33     20306

    accuracy                           0.35     98410
   macro avg       0.34      0.38      0.31     98410
weighted avg       0.53      0.35      0.38     98410



In [51]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=train_data.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

AttributeError: module 'tensorflow_privacy.privacy.analysis.compute_dp_sgd_privacy' has no attribute 'compute_dp_sgd_privacy'